Comparision of charts:

Entire code in Shiny app:
https://monicakherajani.shinyapps.io/appsample/

Using output images in Shiny app:
https://vaidehidhr.shinyapps.io/R_app/

In [ ]:
library(rtweet)
library(twitteR)
library(xlsx)
library(dplyr)
library(stringr)
library(sqldf)

create_token(
  app = "DataClientsForTwitter",
  consumer_key <- "",
  consumer_secret <- "",
  authentication_key <- "",
  authentication_secret <- "")
  
rt <- search_tweets("influenza",geocode = lookup_coords("usa"), n=18000)

rt$symbols <- vapply(rt$symbols, paste, collapse = ", ", character(1L))
rt$hashtags <- vapply(rt$hashtags, paste, collapse = ", ", character(1L))
rt$urls_url <- vapply(rt$urls_url, paste, collapse = ", ", character(1L))
rt$urls_t.co <- vapply(rt$urls_t.co, paste, collapse = ", ", character(1L))
rt$urls_expanded_url <- vapply(rt$urls_expanded_url, paste, collapse = ", ", character(1L))
rt$media_url <- vapply(rt$media_url, paste, collapse = ", ", character(1L))
rt$media_t.co <- vapply(rt$media_t.co, paste, collapse = ", ", character(1L))
rt$media_expanded_url <- vapply(rt$media_expanded_url, paste, collapse = ", ", character(1L))
rt$media_type <- vapply(rt$media_type, paste, collapse = ", ", character(1L))
rt$ext_media_url <- vapply(rt$ext_media_url, paste, collapse = ", ", character(1L))
rt$ext_media_t.co <- vapply(rt$ext_media_t.co, paste, collapse = ", ", character(1L))
rt$ext_media_expanded_url <- vapply(rt$ext_media_expanded_url, paste, collapse = ", ", character(1L))
rt$mentions_user_id <- vapply(rt$mentions_user_id, paste, collapse = ", ", character(1L))
rt$mentions_screen_name <- vapply(rt$mentions_screen_name, paste, collapse = ", ", character(1L))
rt$geo_coords <- vapply(rt$geo_coords, paste, collapse = ", ", character(1L))
rt$coords_coords <- vapply(rt$coords_coords, paste, collapse = ", ", character(1L))
rt$bbox_coords <- vapply(rt$bbox_coords, paste, collapse = ", ", character(1L))
write.csv(rt, 'MyDataforInfluenzaNew.csv')

# Reading the uncleaned tweets obtained with Flu keyword 
fluORInfluenzaKeywordData<-read.csv("MyDataforInfluenzaNew.csv", header = TRUE, sep = ",")
# Removing Re-tweets 
fluORInfluenzaKeywordDataFiltered<-subset(fluORInfluenzaKeywordData,
                                          fluORInfluenzaKeywordData$is_retweet==FALSE)
# Finding the TWO lettered state in User location 
fluORInfluenzaKeywordDataFiltered$location<-str_sub(fluORInfluenzaKeywordDataFiltered$location,start=-2)
# Reading the US states original file 
# Reference:http://www.whypad.com/posts/excel-spreadsheet-of-us-states/583/
USStatesstring <- read.csv("us_states.csv",header = TRUE, sep = ",")
string<-USStatesstring[3]
string <- unlist(string, use.names=FALSE)
# Finding actual state abbreviations in tweet data in User location 
fluORInfluenzaKeywordDataFilteredwithStates<-subset(fluORInfluenzaKeywordDataFiltered, 
                                              fluORInfluenzaKeywordDataFiltered$location %in% string)
# Joining the two dataframes for gettign the enture state name for ploting in Heatmap
finaldataWithfluORInfluenzaKeyword<-sqldf('SELECT * FROM fluORInfluenzaKeywordDataFilteredwithStates 
      LEFT JOIN USStatesstring 
      ON fluORInfluenzaKeywordDataFilteredwithStates.location = USStatesstring.Abbreviation')
# Creating a cleaned CSV file for Flu keyword
write.csv(finaldataWithfluORInfluenzaKeyword, "newInfluenzaKeywordCleanData.csv")



# Reading the uncleaned tweets obtained with Influenza keyword 
influenzaKeywordData<-read.csv("MyDataInfluenza.csv", header = TRUE, sep = ",")
# Removing Re-tweets 
influenzaKeywordDataFiltered<-subset(influenzaKeywordData,influenzaKeywordData$is_retweet==FALSE)
# Finding the TWO lettered state in User location 
influenzaKeywordDataFiltered$location<-str_sub(influenzaKeywordDataFiltered$location,start=-2)
# Reading the US states original file 
# Reference:http://www.whypad.com/posts/excel-spreadsheet-of-us-states/583/
USStatesstring <- read.csv("us_states.csv",header = TRUE, sep = ",")
string<-USStatesstring[3]
string <- unlist(string, use.names=FALSE)
# Finding actual state abbreviations in tweet data in User location 
influenzaKeywordDataFilteredwithStates<-subset(influenzaKeywordDataFiltered, 
                                               influenzaKeywordDataFiltered$location %in% string)
# Joining the two dataframes for gettign the enture state name for ploting in Heatmap
finaldataWithinfluenzaKeyword<-sqldf('SELECT * FROM influenzaKeywordDataFilteredwithStates 
                               LEFT JOIN USStatesstring 
                               ON influenzaKeywordDataFilteredwithStates.location = USStatesstring.Abbreviation')
# Creating a cleaned CSV file for Flu keyword
write.csv(finaldataWithinfluenzaKeyword, "influenzaKeywordCleanData.csv")

# app.R for Entire code in Shiny app:
https://monicakherajani.shinyapps.io/appsample/

In [ ]:
#
# This is a Shiny web application. You can run the application by clicking
# the 'Run App' button above.
#
# Find out more about building applications with Shiny here:
#
#    http://shiny.rstudio.com/
#

library(shiny)
library(shinythemes)
library(dplyr)
library(choroplethr)
library(choroplethrMaps)
library(ggplot2)
library(maptools)
library(ggpubr)


dt1<-read.csv("fluORInfluenzaKeywordCleanData.csv")
flu_data1 <- data.frame(STATENAME=dt1$State)
occurences1 <- data.frame(table(unlist(flu_data1)))
occurences1$Var1 <- tolower(occurences1$Var1)

dt2<-read.csv("influenzaKeywordCleanData.csv")
flu_data2 <- data.frame(STATENAME=dt2$State)
occurences2 <- data.frame(table(unlist(flu_data2)))
occurences2$Var1 <- tolower(occurences2$Var1)

dt3<-read.csv("fluKeywordCleanData2.csv")
flu_data3 <- data.frame(STATENAME=dt3$State)
occurences3 <- data.frame(table(unlist(flu_data3)))
occurences3$Var1 <- tolower(occurences3$Var1)

dt5<-read.csv("StateDataforMap_2018-19week4.csv")
occurences5 <- data.frame(state=dt5$STATENAME, ACTIVITY.LEVEL = dt5$ACTIVITY.LEVEL)
occurences5$state <- tolower(occurences5$state)
occurences5$ACTIVITY.LEVEL <- as.character(occurences5$ACTIVITY.LEVEL)
occurences5$ACTIVITY.LEVEL[occurences5$ACTIVITY.LEVEL == "Level 1"] <- "1"
occurences5$ACTIVITY.LEVEL[occurences5$ACTIVITY.LEVEL == "Level 2"] <- "2"
occurences5$ACTIVITY.LEVEL[occurences5$ACTIVITY.LEVEL == "Level 3"] <- "3"
occurences5$ACTIVITY.LEVEL[occurences5$ACTIVITY.LEVEL == "Level 4"] <- "4"
occurences5$ACTIVITY.LEVEL[occurences5$ACTIVITY.LEVEL == "Level 5"] <- "5"
occurences5$ACTIVITY.LEVEL[occurences5$ACTIVITY.LEVEL == "Level 6"] <- "6"
occurences5$ACTIVITY.LEVEL[occurences5$ACTIVITY.LEVEL == "Level 7"] <- "7"
occurences5$ACTIVITY.LEVEL[occurences5$ACTIVITY.LEVEL == "Level 8"] <- "8"
occurences5$ACTIVITY.LEVEL[occurences5$ACTIVITY.LEVEL == "Level 9"] <- "9"
occurences5$ACTIVITY.LEVEL[occurences5$ACTIVITY.LEVEL == "Level 10"] <- "10"
occurences5$ACTIVITY.LEVEL <- as.numeric(occurences5$ACTIVITY.LEVEL)
colnames(occurences5) <- c("region", "value")

# Define UI for application that draws a histogram
ui <- fluidPage(title = 'Comparisions of Charts',
                # theme = shinythemes::shinytheme('flatly'),
                navlistPanel(
                  "Comparisions",
                  tabPanel('All Keywords VS CDC Heatmap',br(),
                            plotOutput(outputId = "map1"), br(),
                           plotOutput(outputId = "map5")),
                  tabPanel('Influenza VS CDC',br(),
                                plotOutput(outputId = "map2"), br(),
                           plotOutput(outputId = "map6")),
                  tabPanel('Flu VS CDC',br(),
                           plotOutput(outputId = "map3"), br(),
                           plotOutput(outputId = "map7")),
                  tabPanel('Flu VS Influenza',br(),
                           plotOutput(outputId = "map4"), br(),
                           plotOutput(outputId = "map8"))
                
                ))
               
# Define server logic required 
server <- function(input, output) {
  output$map1 <- renderPlot({
    colnames(occurences1) <- c("region", "value")
    state_choropleth(occurences1, title = "All Keywords HeatMap", num_colors = 9, 
                     legend = "Frequency of Tweets")
  })
  output$map2 <- renderPlot({
    colnames(occurences2) <- c("region", "value")
    state_choropleth(occurences2, title = "Influenza Keywords HeatMap", num_colors = 9, 
                     legend = "Frequency of Tweets")
  })
  output$map3 <- renderPlot({
    colnames(occurences3) <- c("region", "value")
    state_choropleth(occurences3, title = "Flu Keywords HeatMap", num_colors = 9, 
                     legend = "Frequency of Tweets")
  })
  output$map4 <- renderPlot({
    colnames(occurences3) <- c("region", "value")
    state_choropleth(occurences3, title = "Flu Keywords HeatMap", num_colors = 9, 
                     legend = "Frequency of Tweets")
  })
  output$map5 <- renderPlot({
    colnames(occurences5) <- c("region", "value")
    state_choropleth(occurences5, title = "CDC Heatmap(Week 4)", num_colors = 9, 
                     legend = "ILI activity level")
  })
  output$map6 <- renderPlot({
    colnames(occurences5) <- c("region", "value")
    state_choropleth(occurences5, title = "CDC Heatmap(Week 4)", num_colors = 9, 
                     legend = "ILI activity level")
  })
  output$map7 <- renderPlot({
    colnames(occurences5) <- c("region", "value")
    state_choropleth(occurences5, title = "CDC Heatmap(Week 4)", num_colors = 9, 
                     legend = "ILI activity level")
  })
  output$map8 <- renderPlot({
    colnames(occurences2) <- c("region", "value")
    state_choropleth(occurences2, title = "Influenza Keywords HeatMap", num_colors = 9, 
                     legend = "Frequency of Tweets")
  })
}

# Run the application 
shinyApp(ui = ui, server = server)

Warning message:
"package 'shiny' was built under R version 3.5.2"Warning message:
"package 'shinythemes' was built under R version 3.5.2"Warning message:
"package 'dplyr' was built under R version 3.5.2"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'choroplethr' was built under R version 3.5.2"Loading required package: acs
Warning message:
"package 'acs' was built under R version 3.5.2"Loading required package: stringr
Warning message:
"package 'stringr' was built under R version 3.5.2"Loading required package: XML
Warning message:
"package 'XML' was built under R version 3.5.2"
Attaching package: 'acs'

The following object is masked from 'package:dplyr':

    combine

The following object is masked from 'package:shiny':

    span

The following object is masked from 'package:base':

    apply

Warning messag

# app.R for Using output images in Shiny app:
https://vaidehidhr.shinyapps.io/R_app/

In [ ]:
#
# This is a Shiny web application. You can run the application by clicking
# the 'Run App' button above.
#
# Find out more about building applications with Shiny here:
#
#    http://shiny.rstudio.com/
#

library(shiny)

# Define UI for application that draws a histogram
ui <- fluidPage(
  
  titlePanel("Lab1-Part3"),
  
  navlistPanel(
    "Comparisions",
    tabPanel("Influenza Vs CDC",
             h6("Influenza Vs CDC", br(), 
                tags$img(src="Capture2.JPG", height=400, width=500),
                tags$img(src="Capture1.JPG",height=400, width=500)
             )
             
    ),
    tabPanel("Flu Vs CDC",
             h6("Flu Vs CDC", br(), 
                tags$img(src="Capture3.JPG", height=400, width=500),
                tags$img(src="Capture1.JPG",height=400, width=500)
             )
             
    ),
    tabPanel("All keywords Vs CDC",
             h6("All keywords Vs CDC", br(), 
                tags$img(src="Capture4.JPG", height=400, width=500),
                tags$img(src="Capture1.JPG",height=400, width=500)
             )
    )
  )
)

# Define server logic required to draw a histogram
server <- function(input, output) {
}

# Run the application 
shinyApp(ui = ui, server = server)